In [56]:
import tkinter as tk

In [57]:
class NaryNode:
    indent = '  '
    node_radius = 10
    x_spacing = 20
    y_spacing = 20

    def __init__(self, value=''):
        self.value = value
        self.children = []
        
        # Initialise drawing parameters
        self.centre = (0, 0)
        self.subtree_bounds = (
            self.centre[0] - self.node_radius,
            self.centre[1] - self.node_radius,
            self.centre[0] + self.node_radius,
            self.centre[1] + self.node_radius
        )

    def __str__(self, level=0):
        return (
            f"{self._indent_node(self.value, level=level)}:\n"
            f"{''.join([child.__str__(level=level+1) for child in self.children])}"
        )

    @classmethod
    def _indent_node(cls, value='', level=0):
        """
        Prepares a string indented to reflect the level of the empty child of the current node. 
        :param value: the text, if available, to include after the indentation 
        :param level: an integer to reflect the level (or depth) in the tree and the indentation required for the node
        :return: string containing enough indentation chars (usually spaces) followed by a text value if available
        """
        return "".join([cls.indent for _ in range(0, level)]) + value

    def add_child(self, child):
        self.children.append(child)

    def find_node(self, value):
        """
        Return the (first) node that has the corresponding value, starting from the current node
        and then traversing child nodes.
        :param value: the value to compare against each node
        :return: the node that matches the given value
        """
        if self.value == value:
            return self
        
        for child in self.children:
            node = child.find_node(value) if child else None
            if node:
                return node
        return None
    
    def traverse_preorder(self):
        nodes = [self]
        for c in self.children:
            nodes.extend(c.traverse_preorder())
        return nodes
    
    def traverse_postorder(self):
        nodes = []
        for c in self.children:
            nodes.extend(c.traverse_postorder())
        nodes.extend([self])
        return nodes
    
    def traverse_breadth_first(self):
        result = []
        queue = [self]
        
        while queue:
            node = queue.pop(0)
            result.append(node)
            
            for c in node.children:
                queue.append(c)
                
        return result
    
    def arrange_subtree(self, xmin, ymin):
        """Position the node's subtree"""
        # Calculate cy, the Y coordinate for this node.
        cy = ymin + self.node_radius
        
        # If the node has no children, just place it here and return
        if not self.children:
            self.centre = (xmin + self.node_radius, cy)
            self.subtree_bounds = (
                xmin,
                ymin,
                self.centre[0] + self.node_radius,
                self.centre[1] + self.node_radius
            )
            return
        
        # Set child_xmin and child_ymin to the
        # start position for child subtrees
        child_xmin = xmin
        child_ymin = cy + self.node_radius + self.y_spacing
        
        # Set ymax equal to the largest Y position used
        ymax = ymin + 2 * self.node_radius
        
        # Position the child subtrees
        for child in self.children:
            # Position this child subtree
            child.arrange_subtree(child_xmin, child_ymin)
    
            # Update child_xmin to allow room for the subtree
            # and space between the subtrees
            child_xmin = child.subtree_bounds[2] + self.x_spacing
    
            # Update the subtree bottom ymax
            ymax = max(ymax, child.subtree_bounds[3])
    
        # Set xmax equal to child_xmin minus the horizontal
        # spacing we added after the last subtree
        xmax = child_xmin - self.x_spacing
    
        # Use xmin, ymin, xmax, ymax to set our subtree bounds
        self.subtree_bounds = (
            xmin,
            ymin,
            xmax,
            ymax
        )
    
        # Centre this node over the subtree bounds
        self.centre = ((xmin + xmax) / 2, ymin + self.node_radius)
    
    def draw_subtree_links(self, canvas):
        """Draw the subtree's links"""
        # If we have exactly one child, just draw to it
        if len(self.children) == 1:
            canvas.create_line(self.centre, self.children[0].centre)
        
        # Else if we have more than one child
        # draw vertical and horizontal branches
        elif len(self.children) > 0:
            # calculate the Y coordinate for the horizontal line
            hy = self.centre[1] + self.node_radius + (self.y_spacing / 2)
            # draw vertical line from this node to horizontal line
            canvas.create_line(self.centre, self.centre[0], hy)
            # draw horizontal line
            canvas.create_line(
                self.children[0].centre[0],
                hy,
                self.children[-1].centre[0],
                hy
            )
        
        # Recursively draw child subtree links
        for child in self.children:
            # draw vertical line to horizontal line (above each child)
            canvas.create_line(
                child.centre,
                child.centre[0],
                child.centre[1] - self.node_radius - (self.y_spacing / 2)
            )
            child.draw_subtree_links(canvas)
        
        # Outline the subtree for debugging
        #canvas.create_rectangle(self.subtree_bounds, fill='', outline='red')
        
    def draw_subtree_nodes(self, canvas):
        """Draw the subtree's nodes"""
        # Draw the node
        canvas.create_oval(
            self.centre[0] - self.node_radius, 
            self.centre[1] - self.node_radius, 
            self.centre[0] + self.node_radius, 
            self.centre[1] + self.node_radius, 
            fill='white',
            outline='green'
        )
        canvas.create_text(self.centre, text=self.value, fill='red')
        
        # Draw the descendants' nodes
        for child in self.children:
            child.draw_subtree_nodes(canvas)
        
    def arrange_and_draw_subtree(self, canvas, xmin, ymin):
        # Position the tree
        self.arrange_subtree(xmin, ymin)
        
        # Draw the links
        self.draw_subtree_links(canvas)
        
        # Draw the nodes
        self.draw_subtree_nodes(canvas)


In [58]:
# Build a test tree.
#         A
#         |
#     +---+---+
#     B   C   D
#     |       |
#    +-+      +
#    E F      G
#    |        |
#    +      +-+-+
#    H      I J K
a = NaryNode('A')
b = NaryNode('B')
c = NaryNode('C')
d = NaryNode('D')
e = NaryNode('E')
f = NaryNode('F')
g = NaryNode('G')
h = NaryNode('H')
i = NaryNode('I')
j = NaryNode('J')
k = NaryNode('K')

a.add_child(b)
a.add_child(c)
a.add_child(d)
b.add_child(e)
b.add_child(f)
d.add_child(g)
e.add_child(h)
g.add_child(i)
g.add_child(j)
g.add_child(k)


In [59]:
def kill_callback():
    """A callback to destroy the tkinter window."""
    window.destroy()


In [60]:
window = tk.Tk()
window.title('nary_node5')
window.protocol('WM_DELETE_WINDOW', kill_callback)
window.geometry('260x180')

canvas = tk.Canvas(window, bg='white', borderwidth=2, relief=tk.SUNKEN)
canvas.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

# Draw the tree
a.arrange_and_draw_subtree(canvas, 10, 10)

window.focus_force()
window.mainloop()